# Introduction


This notebook shows how to run a technical indicator strategy against a live feed coming from the Binance crypto exchange. In this example we're not interested in actually back-testing of the strategy, just the signals it produces.

Whenever there is a new signal, whether it is a BUY or SELL, we get an notfication so we can react to it.

Roboquant includes a dedicated module for crypto support, including Binance that we'll use in this notebook. Since this module already has dependencies declared on the core module, we don't need to inlcude that one also.

In [ ]:
%use roboquant
Welcome()

In [ ]:
import org.roboquant.feeds.binance.*

# Setup
We now create, as usual, the components we want to use. We start with defining a strategy that uses a few technical analysis indicators to detects candlechart patterns and trends in moving averages. This might not be a very useful combination of indicators and just serves to demonstrate how to combine multiple ones.

In [ ]:
val strategy = EMACrossover(3, 5, minEvents = 3)

# Feed
Now we create an instance of the BinanceFeed and subscribe to pricebar events (candlesticks) of the the BTC/BUSD currency pair. It is also possible to subscribe to multiple currency pairs by adding additional parameters to the same *subscribePriceBar* method.

In [ ]:
val feed = BinanceFeed()
feed.subscribePriceBar(Pair("BTCBUSD","BUSD"))

Because we won't be using USD as a base currency, we create a non-default Simbroker with an initial deposit of 100.000,- Binance USD (BUSD). We also define some metrics that will help afterwards to check some statistics:

1) ProgressMetric: overall progress of the run
2) PriceMetric: capture the recorded prices for an asset

In [ ]:
val broker = SimBroker.withDeposit(1_000_000.00, "BUSD")
val priceMetric = PriceMetric(feed.assets.first())
val roboquant =  Roboquant(strategy, ProgressMetric(), priceMetric, broker = broker)

# Live Test
All that remains is to start the run. We run it for 30 minutes, but you can run it of course for any duration.

In [ ]:
val timeFrame = TimeFrame.nextMinutes(30)
roboquant.run(feed, timeFrame)

# Results

In [ ]:
roboquant.logger.summary()

In [ ]:
PriceBarChart(priceMetric, priceMetric.asset)

In [ ]:
PriceChart(priceMetric, priceMetric.asset, roboquant.broker.account.trades)